In [31]:
from datetime import datetime

import numpy as np
import pandas as pd

In [32]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SHOP', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [33]:
DL = '/Users/damiansp/Downloads'

In [34]:
!ls $DL

Portfolio_Positions_Mar-04-2022.csv Positions.csv


In [35]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [36]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

### E*Trade

In [37]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
AMD
AMZN


In [38]:
DONGMEI = False

In [39]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
ABMD,313.72
ABNB,142.7
AMZN,2912.82
ANET,355.65
CHWY,452.70000000000005


In [40]:
check_symbols(et_df)

All clear


In [41]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,314
ABNB,143
AMD,0
AMZN,2913


### Fidelity

In [42]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Mar-04-2022.csv'

In [43]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,CHWY,CHEWY INC CL A,6.0,$45.27,+$1.28,$271.62,+$7.68,+2.90%,+$15.36,+5.99%,0.62%,$256.26,$42.71,Cash
1,218520300,ROLLOVER IRA,ISRG,INTUITIVE SURGICAL INC,3.0,$302.28,+$2.46,$906.84,+$7.38,+0.82%,+$113.58,+14.31%,2.06%,$793.26,$264.42,Cash
2,218520300,ROLLOVER IRA,TTWO,TAKE-TWO INTERACTIVE SOFTWARE INC,2.0,$158.01,-$0.19,$316.02,-$0.38,-0.13%,+$32.62,+11.51%,0.72%,$283.40,$141.70,Cash
3,218520300,ROLLOVER IRA,GMED,GLOBUS MEDICAL INC,4.0,$71.82,-$0.27,$287.28,-$1.08,-0.38%,+$27.68,+10.66%,0.65%,$259.60,$64.90,Cash
4,218520300,ROLLOVER IRA,LULU,LULULEMON ATHLETICA INC COM USD0.005,1.0,$320.58,-$2.26,$320.58,-$2.26,-0.71%,+$16.86,+5.55%,0.73%,$303.72,$303.72,Cash


In [44]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
0,ROLLOVER IRA,CHWY,272
1,ROLLOVER IRA,ISRG,907
2,ROLLOVER IRA,TTWO,316
3,ROLLOVER IRA,GMED,287
4,ROLLOVER IRA,LULU,321


In [45]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
CHWY,272
ISRG,907
TTWO,316
GMED,287
LULU,321


In [46]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [47]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,0,0,1305,0
ABMD,314,0,1569,0
ABNB,143,714,143,0
AMD,0,1409,0,0
AMZN,2913,0,0,0


In [48]:
out.to_csv(f'{DL}/clean.csv')

In [49]:
!open $DL/clean.csv

In [50]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [23]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


In [24]:
mw = pd.read_csv(f'{DL}/Holdings - Damian Satterthwaite-Phillips.csv')[['Symbol', 'Value']]
mw.index = mw.Symbol
mw.drop('Symbol', axis=1, inplace=True)
mw.Value = mw.Value.apply(convert_value)
mw.head()

,Value
Symbol,
ABMD,3137
AMD,1843
AMZN,2913
AXON,1796
BYND,691


In [25]:
check_symbols(mw)

All clear


In [26]:
out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
out.head()

,Value
AAPL,0
ABMD,3137
ABNB,0
AMD,1843
AMZN,2913


In [27]:
out.to_csv(f'{DL}/mw.csv')

In [29]:
!open $DL/mw.csv

In [30]:
!rm $DL/"Holdings - Damian Satterthwaite-Phillips.csv" $DL/mw.csv
!ls $DL

In [15]:
#!rm $DL/*